In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
import pandas as pd

# Full class mapping
signnames = {
    0: "Speed limit (20km/h)",
    1: "Speed limit (30km/h)",
    2: "Speed limit (50km/h)",
    3: "Speed limit (60km/h)",
    4: "Speed limit (70km/h)",
    5: "Speed limit (80km/h)",
    6: "End of speed limit (80km/h)",
    7: "Speed limit (100km/h)",
    8: "Speed limit (120km/h)",
    9: "No passing",
    10: "No passing for vehicles over 3.5 metric tons",
    11: "Right-of-way at the next intersection",
    12: "Priority road",
    13: "Yield",
    14: "Stop",
    15: "No vehicles",
    16: "Vehicles over 3.5 metric tons prohibited",
    17: "No entry",
    18: "General caution",
    19: "Dangerous curve to the left",
    20: "Dangerous curve to the right",
    21: "Double curve",
    22: "Bumpy road",
    23: "Slippery road",
    24: "Road narrows on the right",
    25: "Road work",
    26: "Traffic signals",
    27: "Pedestrians",
    28: "Children crossing",
    29: "Bicycles crossing",
    30: "Beware of ice/snow",
    31: "Wild animals crossing",
    32: "End of all speed and passing limits",
    33: "Turn right ahead",
    34: "Turn left ahead",
    35: "Ahead only",
    36: "Go straight or right",
    37: "Go straight or left",
    38: "Keep right",
    39: "Keep left",
    40: "Roundabout mandatory",
    41: "End of no passing",
    42: "End of no passing by vehicles over 3.5 metric tons"
}

# Save to CSV
df = pd.DataFrame(list(signnames.items()), columns=["ClassId", "SignName"])
df.to_csv("signnames.csv", index=False)

print("✅ signnames.csv created.")


✅ signnames.csv created.


In [ ]:
signnames_df = pd.read_csv("signnames.csv")
signnames_dict = dict(zip(signnames_df.ClassId, signnames_df.SignName))
print("✅ Sign names loaded.")


✅ Sign names loaded.


In [ ]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
!pip install roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 40.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="your_API_key")
project = rf.workspace("gtsrbanno").project("traffic-sign-detection-gtsrb")
version = project.version(13)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to traffic-sign-detection-gtsrb-13 in yolov8:: 100%|██████████| 24465/24465 [00:06<00:00, 3742.28it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import files

# Upload image from your device
uploaded = files.upload()


Saving best (1).pt to best (1).pt


In [ ]:
!mv "best (1).pt" best_yolov8m.pt


In [ ]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 9.4 MB/s eta 0:00:00


In [ ]:
pip install streamlit ultralytics opencv-python-headless


In [ ]:
!pip install pyngrok

In [ ]:
import google.generativeai as genai


In [ ]:
%%writefile app.py
import streamlit as st
from ultralytics import YOLO
import numpy as np
import cv2
from PIL import Image
import google.generativeai as genai
import tempfile
import os
import mimetypes
import io

# --- Streamlit Page Config ---
st.set_page_config(page_title="Traffic Sign Detector", layout="centered")
st.markdown("<h1 style='text-align: center; color: red;'>🚦 Don't know the sign? I'm here to help you! </h1>", unsafe_allow_html=True)

# --- Gemini API Key Input ---
GENAI_API_KEY = st.text_input("🔐 Enter your Gemini API Key", type="password")

# --- Load YOLO Model ---
model = YOLO("best_yolov8m.pt")  # Ensure this model file is in the working directory

if GENAI_API_KEY:
    try:
        genai.configure(api_key=GENAI_API_KEY)
        gemini_model = genai.GenerativeModel("gemini-1.5-flash")
    except Exception as e:
        st.error(f"❌ Failed to configure Gemini API. Error: {e}")
        st.stop()

    # --- Upload Image ---
    uploaded_file = st.file_uploader("📤 Upload a traffic sign image", type=["png", "jpg", "jpeg"])

    if uploaded_file:
        img_bytes = uploaded_file.read()

        # Get mime type early
        mime_type = mimetypes.guess_type(uploaded_file.name)[0]
        if mime_type is None:
            mime_type = "image/jpeg"

        image_part = {
            "mime_type": mime_type,
            "data": img_bytes
        }

        try:
            image = Image.open(io.BytesIO(img_bytes)).convert("RGB")
        except Exception as e:
            st.error(f"❌ Failed to read image. Error: {e}")
            st.stop()

        st.image(image, caption="🖼️ Uploaded Image", use_container_width=True)

        # Save temp image
        temp_dir = tempfile.mkdtemp()
        image_path = os.path.join(temp_dir, uploaded_file.name)
        image.save(image_path)

        # YOLO prediction
        img_np = np.array(image)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
        results = model.predict(source=img_bgr, conf=0.25)
        boxes = results[0].boxes

        if len(boxes) > 0:
            st.success("✅ YOLO detected:")
            for box in boxes:
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                label = model.names[cls_id]
                st.markdown(f"- **{label}** ({conf*100:.1f}% confidence)")

                if conf < 0.5:
                    st.warning(f"⚠️ Low confidence on '{label}'. Asking Gemini for confirmation...")
                    try:
                        response = gemini_model.generate_content([
                            image_part,
                            f"YOLO thinks this is a '{label}' traffic sign with low confidence. Can you confirm if this is correct?"
                        ])
                        st.markdown("### 🤖 Gemini's Confirmation:")
                        st.info(response.text)
                    except Exception as e:
                        st.error(f"❌ Gemini failed to respond. Error: {e}")
        else:
            st.warning("⚠️ YOLO couldn't detect anything confidently.")
            st.markdown("## 🧠 Trying Gemini Vision...")

            try:
                response = gemini_model.generate_content([
                    image_part,
                    "This is a traffic sign. What does it typically mean in road safety?"
                ])
                st.success("🤖 Gemini's Suggestion:")
                st.info(response.text)
            except Exception as e:
                st.error(f"❌ Gemini failed to respond. Error: {e}")

            # Manual fallback
            st.markdown("---")
            st.markdown("📝 Didn't help? Describe the sign manually:")
            user_input = st.text_input("Describe what you see (e.g., red triangle with snowflake):")

            if st.button("🔍 Ask Gemini Again"):
                if user_input.strip():
                    try:
                        response = gemini_model.generate_content([
                            image_part,
                            f"The user says the sign looks like: {user_input}. What does it mean in road safety?"
                        ])
                        st.markdown("### 🤖 Gemini's Refined Answer:")
                        st.success(response.text)
                    except Exception as e:
                        st.error(f"❌ Gemini failed to respond. Error: {e}")
                else:
                    st.warning("⚠️ Please enter a description before submitting.")


Writing app.py


In [ ]:
!ngrok config add-authtoken Your_Auth_token


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!streamlit run app.py &>/content/logs.txt &


In [ ]:
from pyngrok import ngrok

# Correct way (pass port as integer, not string)
public_url = ngrok.connect(8501)  # ✅ NOT "8501"
print(f"🌐 App is live at: {public_url}")


🌐 App is live at: NgrokTunnel: "https://c1f6ec9b49ff.ngrok-free.app" -> "http://localhost:8501"
